In [1]:
%load_ext autoreload
%autoreload 2

$$\begin{aligned}
&\min_X f(X) = \textrm{tr}(A^\top XB X^\top)  \\
& = \textrm{tr}(X^\top A^\top XB) & x = \textrm{vec}(X)\\
& = \left <\textrm{vec}(X),  \textrm{vec}(A^\top X B )  \right > \\
& = \left <\textrm{vec}(X), B^\top \otimes A^\top \cdot \textrm{vec}(X)  \right > \\ 
& = x^\top (B^\top \otimes A^\top) x\\ 
& = \left<AX, XB\right> \\
\mathbf{s.t.} & \\ 
&X \in \Pi_{n}
\end{aligned}$$

In [2]:
from qap_lp.models import *

In [3]:
instance_name = 'chr12a'

A0, B0 = parse(f'qapdata/{instance_name}.dat')
A = A0 / np.linalg.norm(A0)
B = B0 / np.linalg.norm(B0)
n, m = A.shape
e = np.ones(shape=(n, 1))
E = np.ones(shape=(n, n))
ab = np.kron(B.T, A.T)

param = QAPParam(A, B, n, m, e, E, ab)

## projection methods

working on problem:
### exact penalty

$$\begin{aligned}
 &\min_X f  + \mu\cdot | \textrm{tr}(XX^\top) -  n| \\
 &= f  + \mu\cdot n - \mu\cdot \textrm{tr}(XX^\top)  \\
\mathbf{s.t.} & \\
&Xe = X^\top e = e \\
&X \ge 0 \\
\end{aligned}$$

or

$$\begin{aligned}
& \min_X  f  + \mu\cdot n - \mu\cdot \textrm{tr}(XX^\top) - \Lambda \bullet X \\
\mathbf{s.t.} & \\
&Xe = X^\top e = e \\
\end{aligned}$$


very likely to become a concave function, cannot be directly solved by conic solver.



#### project point onto $D_n$

- projection may be computed by mosek
- or some other methods

#### derivatives

$$\begin{aligned}
& \nabla_X F_\mu  = A^\top XB + AXB^\top - 2\mu X \quad (-\Lambda)\\
& \nabla_\mu F_\mu  = n - \textrm{tr}(XX^\top) \\
& \nabla_\Lambda F_\mu  = - X
\end{aligned}$$

#### projected derivative

$$\begin{aligned}
&\min_D ||\nabla F_\mu - D ||_F^2  \\
\mathbf{s.t.} & \\
&D e = D^\top e = 0 \\
\end{aligned}$$


In [73]:
nabla = QAPDerivativeL2Penalty(param)

perm = (7,5,12,2,1,3,9,11,10,6,8,4)
xol = np.zeros((n, n))
for idx, v in enumerate(perm):
    xol[idx, v-1]=1

def gt_min(x):
    return x[x >= -1e-4].min()

# initialize: x0 = [1]/n
x = x0 = np.ones((n, n)) / n
mu = 100
alpha = alpha0 = 0.1
nabla.obj(x, mu), nabla.original_obj(x)

(1100.2610504307938, 0.2610504307937287)

In [ ]:
def proj_on_dn(param, x, **kwargs):
    A, B, n, m, e, E, ab = param.A, param.B, param.n, param.m, param.e, param.E, param.ab
    
    model = mf.Model('projected_on_Dn')
    D = model.variable("d", [*A.shape], dom.inRange(0, 1))
    v = model.variable(1, dom.greaterThan(0.0))
    
    # m = vec(D - dF)
    m = expr.flatten(expr.sub(D, x))
    model.constraint(expr.sum(D, 0), dom.equalsTo(1))
    model.constraint(expr.sum(D, 1), dom.equalsTo(1))
    model.constraint(expr.vstack(v, m), dom.inQCone())
    model.objective(mf.ObjectiveSense.Minimize, v)

    # set params
    userCallback = set_mosek_model_params(model, **kwargs)
    model.setLogHandler(None)
    model.solve()

    model.flushSolutions()
    D_sol = D.level().reshape(A.shape)
    return D_sol

In [ ]:
def point_proj_naive():
    # point projection
    # - stuck at an corner point, vertex.
    # - once reached, it stays.
    for i in range(5):
        obj = nabla.obj(x, mu)
        print(f'====={i }====')
        alpha = 1/(i+1)
        d0 = nabla.partial_f(x, mu)
        x0 = x - d0 * alpha0
        xp = proj_on_dn(param, x0)
    #     # direction
    #     dx = xp - x
    #     st = gt_min(x/dx)
    #     print(f"stepsize: ", alpha, st)
    #     x -=  st * alpha * dx
        x = xp
        print(f"obj: ", nabla.original_obj(x), nabla.obj(x, mu), nabla.obj(x, mu) - obj)
        print(f"stopping: ", n - x.dot(x.T).trace())
        print(x.round(3).min())
        print(np.linalg.norm(d0))

#### gradient projection

- projection may be computed by mosek
- or some other methods

In [5]:
# dF = x
def pd_on_dc(param, dF, **kwargs):

    A, B, n, m, e, E, ab = param.A, param.B, param.n, param.m, param.e, param.E, param.ab

    model = mf.Model('projected_gradient_on_D_cone')
    D = model.variable("d", [*A.shape], dom.unbounded())
    v = model.variable(1, dom.greaterThan(0))

    # m = vec(D - dF)
    m = expr.flatten(expr.sub(D, dF))
    model.constraint(expr.sum(D, 0), dom.equalsTo(0))
    model.constraint(expr.sum(D, 1), dom.equalsTo(0))
    model.constraint(expr.vstack(v, m), dom.inQCone())
    # model.constraint(expr.sub(), dom.greaterThan(0))
    model.objective(mf.ObjectiveSense.Minimize, v)

    # set params
    userCallback = set_mosek_model_params(model, **kwargs)
    model.setLogHandler(None)
    model.solve()

    model.flushSolutions()
    D_sol = D.level().reshape(A.shape)
    return D_sol

In [52]:
def st(dp, x):
    model = mf.Model('step_size')
    v = model.variable(1, dom.greaterThan(0))
    V = expr.reshape(expr.repeat(v, n*n, 0), [n, n])
    delta = expr.mulElm(V, dp)
    model.constraint(expr.sub(x, delta), dom.greaterThan(0))
    model.objective(mf.ObjectiveSense.Maximize, v)
#     model.setLogHandler(sys.stdout)
    try:
        model.solve()
        return v.level()[0]
    except:
        return 0

In [74]:
L = np.ones((n, n))
alpha = 0.1
# gradient projection, not good for now
for i in range(20):
    print(f'====={i }====')
    obj = nabla.obj(x, mu)
    alpha *= 0.9
    d0 = nabla.partial_f(x, mu) 
    dp = pd_on_dc(param, d0)
    ndf = np.linalg.norm(dp)
    stp = st(dp, x)
    print(f"norm: {ndf}")
    x -=  stp * dp * alpha
    print(f"obj: ", nabla.obj(x, mu), obj)
    print(f"stopping: ", n - x.dot(x.T).trace())
    if stp == 0:
        break
    print(x.round(3).min())

=====0====
norm: 0.040302978366816275
obj:  1100.1833861271564 1100.2610504307938
stopping:  10.999234626342345
0.076
=====1====
norm: 5.574198898752168
obj:  1100.0057859437654 1100.1833861271564
stopping:  10.99746803073209
0.07
=====2====
norm: 10.105586686953613
obj:  1099.7813523977552 1100.0057859437654
stopping:  10.995231595980718
0.065
=====3====
norm: 13.853171480013962
obj:  1099.5403689094492 1099.7813523977552
stopping:  10.992828437255374
0.06
=====4====
norm: 16.979954625384092
obj:  1099.299888988299 1099.5403689094492
stopping:  10.990429311500833
0.057
=====5====
norm: 19.609332083915273
obj:  1099.0692150813766 1099.299888988299
stopping:  10.988127419310093
0.054
=====6====
norm: 21.835977961476065
obj:  1098.853085515799 1099.0692150813766
stopping:  10.985970284632911
0.051
=====7====
norm: 23.733421923184444
obj:  1098.6535549563994 1098.853085515799
stopping:  10.983978567224034
0.049
=====8====
norm: 25.359417521898866
obj:  1098.471116242286 1098.6535549563994

In [77]:
dp

array([[-1.08568437,  4.22437648,  6.75583483, -2.34597504, -1.40192071,
         3.11390094, -5.91158091, -0.37246223,  0.66160016, -4.56742772,
        -7.17490839,  8.10424695],
       [-1.23431467,  4.79304272,  7.66281032, -2.6587959 , -1.59266053,
         3.53185248, -6.70538983, -0.42480268,  0.75186048, -5.18213382,
        -8.13567946,  9.19421089],
       [ 0.29961022, -1.15817991, -1.85028025,  0.64083753,  0.38594845,
        -0.85275824,  1.61919437,  0.10384916, -0.18233123,  1.25212191,
         1.96311416, -2.22112618],
       [ 0.64778745, -2.52429916, -4.03794908,  1.40301578,  0.83693603,
        -1.86120741,  3.53327143,  0.22170664, -0.39487392,  2.72934553,
         4.28939115, -4.84312444],
       [ 0.40386713, -1.57933468, -2.52776693,  0.87951223,  0.52247298,
        -1.16517435,  2.21173729,  0.13744874, -0.24636681,  1.7077034 ,
         2.68658481, -3.03068382],
       [-0.15827648,  0.5949185 ,  0.94608   , -0.32391242, -0.20181236,
         0.43586697, -

# eval

In [ ]:
x_int, _ = extract_sol_rounding(x, param.A, param.B)

In [ ]:
def check_obj_val(x_sol):
    _obj = A0.T.dot(x_sol).dot(B0).dot(x_sol.T).trace()
    print(f'original obj {_obj}')
    return _obj

In [ ]:
check_obj_val(x)

In [ ]:
nabla.original_obj(xol), nabla.obj(xol, mu)